In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import warnings

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv1D, MaxPooling1D

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

2024-02-13 17:54:31.974792: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 17:54:31.975000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 17:54:32.164797: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/biomed-datathon-bmefest2/train.csv')
df_test = pd.read_csv('/kaggle/input/biomed-datathon-bmefest2/test_files.csv')
df.head()

,patient_id,AS,AR,MR,MS,N,recording_1,recording_2,recording_3,recording_4,recording_5,recording_6,recording_7,recording_8
0,patient_052,0,1,0,0,0,052_sup_Mit,052_sup_Tri,052_sup_Pul,052_sup_Aor,052_sit_Mit,052_sit_Tri,052_sit_Pul,052_sit_Aor
1,patient_058,0,1,0,0,0,058_sup_Mit,058_sup_Tri,058_sup_Pul,058_sup_Aor,058_sit_Mit,058_sit_Tri,058_sit_Pul,058_sit_Aor
2,patient_068,0,1,0,0,0,068_sup_Mit,068_sup_Tri,068_sup_Pul,068_sup_Aor,068_sit_Mit,068_sit_Tri,068_sit_Pul,068_sit_Aor
3,patient_074,0,1,0,0,0,074_sup_Mit,074_sup_Tri,074_sup_Pul,074_sup_Aor,074_sit_Mit,074_sit_Tri,074_sit_Pul,074_sit_Aor
4,patient_054,1,0,0,0,0,054_sup_Mit,054_sup_Tri,054_sup_Pul,054_sup_Aor,054_sit_Mit,054_sit_Tri,054_sit_Pul,054_sit_Aor


In [3]:
df_test.head()

,patient_id,recording_1,recording_2,recording_3,recording_4,recording_5,recording_6,recording_7,recording_8
0,patient_001,001_sit_Aor,001_sit_Mit,001_sit_Pul,001_sit_Tri,001_sup_Aor,001_sup_Mit,001_sup_Pul,001_sup_Tri
1,patient_003,003_sit_Aor,003_sit_Mit,003_sit_Pul,003_sit_Tri,003_sup_Aor,003_sup_Mit,003_sup_Pul,003_sup_Tri
2,patient_005,005_sit_Aor,005_sit_Mit,005_sit_Pul,005_sit_Tri,005_sup_Aor,005_sup_Mit,005_sup_Pul,005_sup_Tri
3,patient_007,007_sit_Aor,007_sit_Mit,007_sit_Pul,007_sit_Tri,007_sup_Aor,007_sup_Mit,007_sup_Pul,007_sup_Tri
4,patient_008,008_sit_Aor,008_sit_Mit,008_sit_Pul,008_sit_Tri,008_sup_Aor,008_sup_Mit,008_sup_Pul,008_sup_Tri


In [4]:
X = []
y = []

max_time_steps = 118 

for index, row in df.iterrows():

    patient_features = []
    for i in range(1, 9):  
        recording_column = f'recording_{i}'
        if row[recording_column] == "085_sit_Tri":
            file_path = f"/kaggle/input/biomed-datathon-bmefest2/train/085_sit_Tri6_06.wav"
        else:
            file_path = f"/kaggle/input/biomed-datathon-bmefest2/train/{row[recording_column]}.wav"

        g, sr = librosa.load(file_path, sr=None)

        features = librosa.feature.mfcc(y=g, sr=sr, n_mfcc=13)

        features_padded = pad_sequences([features.T], maxlen=max_time_steps, padding='post', dtype='float32', truncating='post', value=-999.0).T

        features_padded = np.squeeze(features_padded)

        patient_features.append(features_padded)

    X.append(patient_features)

    label_vector = [row["AS"], row["AR"], row["MR"], row["MS"], row["N"]]
    y.append(label_vector)

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (59, 8, 13, 118)
y shape: (59, 5)


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   patient_id   49 non-null     object
 1   recording_1  49 non-null     object
 2   recording_2  49 non-null     object
 3   recording_3  49 non-null     object
 4   recording_4  49 non-null     object
 5   recording_5  49 non-null     object
 6   recording_6  49 non-null     object
 7   recording_7  49 non-null     object
 8   recording_8  49 non-null     object
dtypes: object(9)
memory usage: 3.6+ KB


In [6]:
X_test = []

max_time_steps = 118  

for index, row in df_test.iterrows():

    patient_features = []
    for i in range(1, 9):  
        recording_column = f'recording_{i}'
        file_path = f"/kaggle/input/biomed-datathon-bmefest2/test/{row[recording_column]}.wav"

        g, sr = librosa.load(file_path, sr=None)

        features = librosa.feature.mfcc(y=g, sr=sr, n_mfcc=13)

        features_truncated = features[:, :max_time_steps]

        patient_features.append(features_truncated)

    X_test.append(patient_features)

X_test = np.array(X_test)

print("X_test shape:", X_test.shape)


X_test shape: (49, 8, 13, 118)


In [7]:
X = np.array(X)
X_test = np.array(X_test)
y = df[['AS','AR','MR','MS','N']]
y = np.array(y)
scaler = StandardScaler()
X = scaler.fit_transform(X.reshape((X.shape[0], -1)))
X_test = scaler.fit_transform(X_test.reshape((X_test.shape[0], -1)))

print("X shape:", X.shape)
print("y shape:", y.shape)
print("X_test shape:", X_test.shape)

X shape: (59, 12272)
y shape: (59, 5)
X_test shape: (49, 12272)


In [8]:
model = Sequential()
model.add(Flatten(input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# Training the model
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.1)

Epoch 1/100
2/2 [==============================] - 1s 265ms/step - loss: 0.8558 - accuracy: 0.1509 - val_loss: 0.9783 - val_accuracy: 0.5000
Epoch 2/100
2/2 [==============================] - 0s 54ms/step - loss: 0.3090 - accuracy: 0.6038 - val_loss: 1.1066 - val_accuracy: 0.1667
Epoch 3/100
2/2 [==============================] - 0s 50ms/step - loss: 0.1326 - accuracy: 0.7925 - val_loss: 1.1416 - val_accuracy: 0.1667
Epoch 4/100
2/2 [==============================] - 0s 46ms/step - loss: 0.0152 - accuracy: 0.8113 - val_loss: 1.1046 - val_accuracy: 0.5000
Epoch 5/100
2/2 [==============================] - 0s 43ms/step - loss: 0.0111 - accuracy: 0.7547 - val_loss: 1.1301 - val_accuracy: 0.5000
Epoch 6/100
2/2 [==============================] - 0s 64ms/step - loss: 0.0023 - accuracy: 0.7547 - val_loss: 1.1643 - val_accuracy: 0.5000
Epoch 7/100
2/2 [==============================] - 0s 47ms/step - loss: 0.0014 - accuracy: 0.7547 - val_loss: 1.1931 - val_accuracy: 0.5000
Epoch 8/100
2/2 [==

In [10]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # Threshold probabilities to get binary predictions

2/2 [==============================] - 0s 6ms/step


In [11]:
y_pred
DF = pd.DataFrame(y_pred)
ID = pd.DataFrame(df_test['patient_id'])
data = pd.concat([ID, DF], axis=1)
data.to_csv("Y_BioMed.csv")

In [12]:
X_test.shape

(49, 12272)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patient_id  49 non-null     object
 1   0           49 non-null     int64 
 2   1           49 non-null     int64 
 3   2           49 non-null     int64 
 4   3           49 non-null     int64 
 5   4           49 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.4+ KB
